In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import sys
sys.path.append("../graphsage/")
import 

In [3]:
from utils import load_data

In [7]:
import numpy as np

In [137]:
from sklearn.metrics import accuracy_score

In [8]:
G, feats, id_map, _, class_map = load_data("/data/ppi/ppi")

Removed 0 nodes that lacked proper annotations due to networkx versioning issues
Loaded data.. now preprocessing..


In [98]:
train_X, train_y = [], []
test_X, test_y = [], []
val_X, val_y = [], []

for nodeid in G.nodes():
    idx = id_map[nodeid]
    c = class_map[nodeid]
    if G.node[nodeid]['test']:
        test_X.append(feats[idx])
        test_y.append(c)
    elif G.node[nodeid]['val']:
        val_X.append(feats[idx])
        val_y.append(c)
    else:
        train_X.append(feats[idx])
        train_y.append(c)
        
train_X = np.array(train_X, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)
test_X = np.array(test_X, dtype=np.float32)

train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())
train_y = Variable(torch.Tensor(train_y).long())
test_y = Variable(torch.Tensor(test_y).long())

In [100]:
train_y[:,0]

tensor([1, 1, 1,  ..., 1, 1, 1])

In [155]:
all_preds = []

In [ ]:
class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self ).__init__()
        self.fc1 = nn.Linear(50, 100)
        self.fc3 = nn.Linear(100, 2)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc3(X)
        #X = F.sigmoid(X)

        return X
    
for i in range(121):
    train_y_ = train_y[:,i]
    test_y_ = test_y[:,i]

    net = Net()

    criterion = nn.CrossEntropyLoss()# cross entropy loss
    
    optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
    
    for epoch in range(1000):
        optimizer.zero_grad()
        out = net(train_X)
        loss = criterion(out, train_y_)
        loss.backward()
        optimizer.step()
    
    pred = np.array(net(test_X).detach())
    pred = np.argmax(pred, axis=1)
    all_preds.append(pred)
    print("i: ", i, f1_score(y_pred=pred, y_true=np.array(test_y_), average="micro"))
    print("i: ", i, f1_score(y_pred=pred, y_true=np.array(test_y_), average="macro"))

('i: ', 0, 0.6417451122375091)
('i: ', 0, 0.4093479208385447)
('i: ', 1, 0.7659304851556843)
('i: ', 1, 0.4352411278300776)
('i: ', 2, 0.7939898624185373)
('i: ', 2, 0.4425832492431887)


In [153]:
%pdb

Automatic pdb calling has been turned ON


In [112]:
len(all_preds)

242

In [116]:
all_preds = all_preds[:121] 

In [121]:
all_preds = np.array(all_preds)

In [133]:
all_preds[0].shape

(5524,)

In [122]:
all_preds[:,0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [141]:
f1_score(np.array(test_y), all_preds.T, average="micro")

0.43322728879462713

In [140]:
f1_score(y_pred=all_preds.T, y_true=np.array(test_y), average="macro")

0.15738736881868182

In [148]:
mac_acc = 0
for i in range(121):
    mac_acc += f1_score(y_pred=all_preds[i], y_true=np.array(test_y)[:,i], average="macro")

In [ ]:
mac_acc / 120

In [135]:
f1_score(y_pred=all_preds[0], y_true=np.array(test_y)[:,0], average="macro")

0.3900801973819237

In [136]:
f1_score(y_pred=all_preds[1], y_true=np.array(test_y)[:,1], average="micro") 

0.7662925416364953

In [144]:
f1_score(np.ravel(np.array(test_y)), np.ravel(all_preds.T), average="micro")

0.7481208969425677

In [145]:
f1_score(np.ravel(np.array(test_y)), np.ravel(all_preds.T), average="macro")

0.6356543296303908

In [151]:
def micro_f1(logits, labels): 
    # Compute predictions 
    preds = torch.round(nn.Sigmoid()(logits)) 
    # Cast to avoid trouble 
    preds = preds.long() 
    labels = labels.long() 
    # Count true positives, true negatives, false positives, false negatives 
    tp = torch.nonzero(preds * labels).shape[0] * 1.0 
    tn = torch.nonzero((preds - 1) * (labels - 1)).shape[0] * 1.0 
    fp = torch.nonzero(preds * (labels - 1)).shape[0] * 1.0 
    fn = torch.nonzero((preds - 1) * labels).shape[0] * 1.0 
    # Compute micro-f1 score 
    prec = tp / (tp + fp) 
    rec = tp / (tp + fn) 
    f1 = (2 * prec * rec) / (prec + rec) 
    return f1